# データサイエンス100本ノック 71~80

In [1]:
import polars as pl
from helper.polars import load_100knocks_data
pl.Config.set_fmt_str_lengths(100)
df_customer, df_category, df_product, df_receipt, df_store, df_geocode = load_100knocks_data()

## P-071

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [27]:
(
df_receipt
.select('customer_id', 'sales_ymd')
.unique(keep='first')
.join(df_customer, on='customer_id')
.select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
)
.with_columns(
    elapsed_months=
        (pl.col.sales_ymd.dt.year() - pl.col.application_date.dt.year()) * 12 +
        (pl.col.sales_ymd.dt.month() - pl.col.application_date.dt.month())
)
.head()
)

customer_id,sales_ymd,application_date,elapsed_months
str,date,date,i32
"""CS025115000002""",2019-09-13,2016-01-16,44
"""CS025304000004""",2017-04-16,2016-01-04,15
"""CS029415000098""",2019-09-09,2015-07-13,50
"""CS027414000125""",2019-07-20,2015-10-15,45
"""CS017315000003""",2019-09-17,2015-07-02,50


## P-072

レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [28]:
(
df_receipt
.select('customer_id', 'sales_ymd')
.unique(keep='first')
.join(df_customer, on='customer_id')
.select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
)
.with_columns(
    elapsed_years=pl.col.sales_ymd.dt.year() - pl.col.application_date.dt.year()
)
.head()
)

customer_id,sales_ymd,application_date,elapsed_years
str,date,date,i32
"""CS038315000124""",2017-09-17,2015-04-25,2
"""CS022515000128""",2018-02-08,2016-01-16,2
"""CS045615000002""",2019-08-08,2017-05-10,2
"""CS016414000063""",2019-06-17,2014-11-20,5
"""CS008414000023""",2019-06-12,2015-05-21,4


## P-073

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [29]:
(
df_receipt
.select('customer_id', 'sales_ymd')
.unique(keep='first')
.join(df_customer, on='customer_id')
.select(
    pl.col.customer_id,
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
    pl.col.application_date.cast(str).str.strptime(pl.Date, '%Y%m%d')
)
.with_columns(
    elapsed_years=(pl.col.sales_ymd - pl.col.application_date).dt.total_seconds()
)
.head()
)

customer_id,sales_ymd,application_date,elapsed_years
str,date,date,i64
"""CS020414000028""",2017-03-02,2015-04-04,60307200
"""CS039215000046""",2017-03-23,2015-08-20,50198400
"""CS035414000064""",2017-08-10,2015-03-21,75427200
"""CS019414000012""",2018-02-01,2015-03-22,90460800
"""CS035414000071""",2017-01-29,2014-10-19,71971200


## P-074

レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [30]:
(
df_receipt
.select('customer_id', 'sales_ymd')
.join(df_customer, on='customer_id')
.select(
    pl.col.sales_ymd.cast(str).str.strptime(pl.Date, '%Y%m%d'),
)
.with_columns(
    pl.col.sales_ymd.dt.truncate('1w').alias('monday')
)
.with_columns(
    (pl.col.sales_ymd - pl.col.monday).alias('elapsed_days').dt.total_days()
)
.head()
)

sales_ymd,monday,elapsed_days
date,date,i64
2018-11-03,2018-10-29,5
2018-11-18,2018-11-12,6
2017-07-12,2017-07-10,2
2018-08-21,2018-08-20,1
2019-06-05,2019-06-03,2


## P-075

顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [31]:
df_customer.sample(fraction=0.01).head(5)

customer_id,customer_name,gender_cd,gender,birth_day,age,postal_cd,address,application_store_cd,application_date,status_cd
str,str,str,str,str,i64,str,str,str,i64,str
"""CS017713000064""","""西谷 りえ""","""1""","""女性""","""1947-12-15""",71,"""165-0027""","""東京都中野区野方**********""","""S13017""",20150306,"""0-00000000-0"""
"""CS026415000057""","""宮崎 あさみ""","""9""","""不明""","""1974-12-29""",44,"""253-0013""","""神奈川県茅ヶ崎市赤松町**********""","""S14026""",20150401,"""E-20100916-D"""
"""CS027211000005""","""吉井 華子""","""1""","""女性""","""1995-11-20""",23,"""245-0067""","""神奈川県横浜市戸塚区深谷町**********""","""S14027""",20141101,"""6-20100831-9"""
"""CS030514000050""","""真田 真帆""","""1""","""女性""","""1965-06-19""",53,"""272-0031""","""千葉県市川市平田**********""","""S12030""",20150703,"""B-20100729-A"""
"""CS011705000002""","""島津 哲平""","""0""","""男性""","""1940-03-05""",79,"""223-0065""","""神奈川県横浜市港北区高田東**********""","""S14011""",20150812,"""0-00000000-0"""


## P-076

顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [32]:
(
df_customer
.filter(
    (pl.int_range(pl.len()) < pl.len() * 0.1).shuffle().over('gender_cd') == 1
)
.group_by('gender_cd')
.agg(pl.len())
)

gender_cd,len
str,u32
"""9""",108
"""1""",1792
"""0""",299


## P-077

レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [33]:
(
df_receipt
.group_by('customer_id')
.agg(
    sum_amount=pl.col.amount.sum(),
    log_sum_amount=pl.col.amount.sum().log(),
)
.filter(
    (pl.col.log_sum_amount - pl.col.log_sum_amount.mean()) / pl.col.log_sum_amount.std() > 3
)
)

customer_id,sum_amount,log_sum_amount
str,i64,f64
"""ZZ000000000000""",12395003,16.332804


## P-078

レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [34]:
col = pl.col.sum_amount
q25 = col.quantile(0.25)
q75 = col.quantile(0.75)
iqr = q75 - q25
expr = (col > q75 + iqr * 1.5) | (col < q25 - iqr * 1.5)
(
df_receipt
.filter(
    pl.col.customer_id.str.starts_with('Z').not_()
)
.group_by('customer_id')
.agg(sum_amount=pl.col.amount.sum())
.filter(expr)
.head()
)

customer_id,sum_amount
str,i64
"""CS026515000201""",11144
"""CS007615000046""",8979
"""CS021515000101""",10401
"""CS022515000119""",8354
"""CS025414000027""",8603


## P-079

商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [35]:
df_product.select(
    pl.all().null_count()
)

product_cd,category_major_cd,category_medium_cd,category_small_cd,unit_price,unit_cost
u32,u32,u32,u32,u32,u32
0,0,0,0,7,7


## P-080

商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [36]:
len(df_product), len(df_product.drop_nulls())

(10030, 10023)